## HW4 - Zhamilya Saparova


In [81]:
pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [82]:
pip install ir_measures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time
import pandas as pd
import numpy as np
import ir_measures
from ir_measures import *

In [84]:
es = Elasticsearch("http://localhost:9200")
ELASTIC_PASSWORD = 'OsTuAI29lih200BZT70Pp4Sy'

CLOUD_ID = 'ir_hw4:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRiYzA3ZDRkMGRhYjc0OGNmOGQyZGE0ZjEzZDYyMTY5OCQ4ZDEyYTI0ZGU5Yzc0ZmI4YTViMmFlZWU4M2M5NDcxMQ=='
es = Elasticsearch(cloud_id = CLOUD_ID, basic_auth=('elastic', ELASTIC_PASSWORD))

print("Connection to ES Server successful")

Connection to ES Server successful


In [85]:
es.indices.delete(index='hello')

ObjectApiResponse({'acknowledged': True})

In [86]:
es.indices.create(index='hello')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hello'})

In [87]:
settings_final = {
    'mappings': {
        'properties': {
            'id_right': {
                'type': 'integer'
            },
            'text_right': {
                'type': 'text',
                'fields': {
                    'white':{
                        'type': 'text',
                        'analyzer': 'white_analyzer'
                    }
                }
            }
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'white_analyzer':{
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase',
                        'snowball_eng'
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace',
                }
            },
            'filter': {
                'snowball_eng':{
                    'type': 'snowball',
                    'language': 'english'
                }
            }
        }
    }
    
}

In [88]:
settings_final_nostem = {
    'mappings': {
        'properties': {
            'id_right': {
                'type': 'integer'
            },
            'text_right': {
                'type': 'text',
                'fields': {
                    'white':{
                        'type': 'text',
                        'analyzer': 'white_analyzer'
                    }
                }
            },
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'white_analyzer':{
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase',
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace',
                }
            },
        }
    }
}

In [89]:
def recreate_index():
    es.indices.delete(index='hello')
    es.indices.create(index='hello', body = settings_final)

In [90]:
def recreate_index_nostem():
    es.indices.delete(index='hello')
    es.indices.create(index='hello', body = settings_final_nostem)

In [91]:
def check_analyzer(analyzer, text):
    body = analyzer
    body['text'] = text
    
    tokens = es.indices.analyze(index='hello', body=body)['tokens']
    tokens = [token_info['token'] for token_info in tokens]
    return tokens

In [92]:
recreate_index()

<ipython-input-89-5d77653c0115>:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='hello', body = settings_final)


In [93]:
# recreate_index_nostem()

## INDEX DOCUMENTS

In [94]:
doc_df = pd.read_csv('/content/documents.csv')
doc = doc_df.to_dict('records')

In [95]:
type(doc), len(doc), type(doc_df), doc_df.shape

(list, 369721, pandas.core.frame.DataFrame, (369721, 2))

In [96]:
doc_df.head()

,id_right,text_right
0,1781133,it was used in landing craft during world war ...
1,2426736,after rejecting an offer from cambridge univer...
2,2224122,mat zan coached kuala lumpur fa in 1999 and wo...
3,219642,a barcode is a machine readable optical label ...
4,1728654,since the subordination of the monarchy under ...


In [97]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

In [98]:
def es_actions_generator():
    for doc_id in doc:
        yield create_es_action('hello', doc_id['id_right'], doc_id)

In [99]:
# ADDING DOCS TO THE INDEX
%%time 
for ok, result in parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000):
    if not ok:
        print(result)

ConnectionTimeout: ignored

## SEARCH


In [100]:
def search(query, *args):
    pretty_print_result(es.search(index='hello', body=query, size=20), args)
                        
def pretty_print_result(search_result, fields=[]):
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')
                  
def get_doc_by_id(doc_id):
    return es.get(index='hello', id=doc_id)['_source']

In [101]:
%%time 

query = {
    'query': {
        'match_all': {}
    }
}
search(query)

<ipython-input-100-fae58037f7d9>:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  pretty_print_result(es.search(index='hello', body=query, size=20), args)


Total documents: 10000
Doc 1781133, score is 1.0
Doc 2426736, score is 1.0
Doc 2224122, score is 1.0
Doc 219642, score is 1.0
Doc 1728654, score is 1.0
Doc 1889917, score is 1.0
Doc 1518473, score is 1.0
Doc 1459418, score is 1.0
Doc 1102956, score is 1.0
Doc 931408, score is 1.0
Doc 1043905, score is 1.0
Doc 488327, score is 1.0
Doc 1069841, score is 1.0
Doc 1601336, score is 1.0
Doc 1415191, score is 1.0
Doc 1531137, score is 1.0
Doc 912052, score is 1.0
Doc 935016, score is 1.0
Doc 1671729, score is 1.0
Doc 1946935, score is 1.0
CPU times: user 72.8 ms, sys: 3.17 ms, total: 76 ms
Wall time: 2.76 s


In [102]:
%%time

query = {
    'query': {
        'bool': {
            'must': {
                'match': {
                    'text_right': 'private boats'
                }
            }
        }
    }
}

search(query)

Total documents: 10000
Doc 2355214, score is 13.976819
Doc 644717, score is 13.686264
Doc 1596321, score is 13.1172695
Doc 2412578, score is 12.66897
Doc 352785, score is 12.558475
Doc 1272069, score is 12.242398
Doc 1226233, score is 12.007725
Doc 1856656, score is 12.007725
Doc 880419, score is 12.007725
Doc 919902, score is 12.007725
Doc 1879751, score is 12.007725
Doc 1110284, score is 11.860317
Doc 2029858, score is 10.934549
Doc 1964437, score is 10.934549
Doc 1329021, score is 10.934549
Doc 1714910, score is 10.934549
Doc 551366, score is 10.934549
Doc 885487, score is 10.934549
Doc 2075473, score is 10.934549
Doc 1185710, score is 10.884844
CPU times: user 63 ms, sys: 3.12 ms, total: 66.2 ms
Wall time: 171 ms


<ipython-input-100-fae58037f7d9>:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  pretty_print_result(es.search(index='hello', body=query, size=20), args)


In [103]:
pip install -U sentence-transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [104]:
import datetime
import time
import torch
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

q = pd.read_csv('/content/queries.csv')
q = q.to_dict('records')
runs = dict()

for i in range(len(q)):
  st = time.time()
  queries = dict()
  query = {
        'query': {
            'bool': {
                'must': {
                    'match': {
                        'text_right': q[i]['text_left']
                    }
                }
            }
        }
    }
  search_results = es.search(index = 'hello', body = query, size = 20)
 
  corpus = []
  idx = dict()

  if len(search_results['hits']['hits']) != 0:


    for j in range(len(search_results['hits']['hits'])):
      corp = search_results['hits']['hits'][j]['_source']['text_right']
      idx[j] = search_results['hits']['hits'][j]['_source']['id_right']
      corpus.append(corp)
    
    top_k = min(20, len(corpus))
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    query_embedding = embedder.encode(q[i]['text_left'], convert_to_tensor = True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    for scoree, idxx in zip(top_results[0], top_results[1]):
      doc_id = str(idx[int(idxx)])
      score = scoree
      query_id = i
      queries[doc_id] = score
    
    runs[str(q[i]['id_left'])] = queries

    et = time.time()
    elapsed_time = et - st
    print(query_id, '\t', elapsed_time)


<ipython-input-104-a506ce4805d7>:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_results = es.search(index = 'hello', body = query, size = 20)


0 	 4.581221580505371
1 	 3.860698938369751
2 	 6.051204204559326
3 	 3.524930715560913
4 	 2.7110519409179688
5 	 3.0900495052337646
6 	 3.4311184883117676
7 	 2.9274957180023193
8 	 2.909024953842163
9 	 2.9369266033172607
10 	 3.763282060623169
11 	 2.7413384914398193
12 	 2.9902751445770264
13 	 2.9374516010284424
15 	 3.4219930171966553
16 	 2.931328296661377
17 	 2.878676652908325
18 	 2.75605845451355
19 	 3.0240275859832764
20 	 3.0322694778442383
21 	 2.6670000553131104
22 	 2.6793267726898193
23 	 2.986239433288574
24 	 3.825251579284668
25 	 2.5643725395202637
26 	 2.449552297592163
27 	 2.9595181941986084
28 	 3.842911958694458
29 	 2.927267074584961
30 	 2.7319977283477783
31 	 2.9829471111297607
32 	 3.7581000328063965
33 	 3.0230772495269775
34 	 2.639892816543579
35 	 2.855710029602051
36 	 3.144117593765259
37 	 2.665174961090088
38 	 2.9656708240509033
39 	 2.9406049251556396
40 	 3.005831718444824
41 	 3.0152902603149414
42 	 2.6778383255004883
43 	 2.767098188400268

## PART 2 EVALUATION

In [105]:
#  Qrel(query_id='13540', doc_id='1276852', relevance=1, iteration='0'),

qrels = ir_measures.read_trec_qrels('/content/qrels')
qrels_ = dict()
qrels_new = dict()
for i in list(qrels):
  query_id = i[0]
  doc_id = i[1]
  relevance = i[2]
  qrels_[doc_id] = relevance
  qrels_new[query_id] = qrels_

In [106]:
import datetime
import time
q = pd.read_csv('/content/queries.csv')
q = q.to_dict('records')
runs = dict()
for i in range(len(q)):
  st = time.time()
  queries = dict()
  query = {
        'query': {
            'bool': {
                'must': {
                    'match': {
                        'text_right': q[i]['text_left']
                    }
                }
            }
        }
    }
  search_results = es.search(index = 'hello', body = query, size = 20)
  res = search_results['hits']
  for hit in res['hits']:
    doc_id = str(hit['_id'])
    score = float(hit['_score'])
    query_id = i
    queries[doc_id] = score

  runs[str(q[i]['id_left'])] = queries
  
scores = ir_measures.calc_aggregate([P(rel=1)@10, P(rel=1)@20, AP], qrels_new, runs)
et = time.time()
elapsed_time = et - st
print(scores)
print('TIME', elapsed_time)

<ipython-input-106-858307cd7779>:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_results = es.search(index = 'hello', body = query, size = 20)


{AP: 0.0004083007253942143, P@20: 0.16450000000000006, P@10: 0.22599999999999987}
TIME 0.36681342124938965


In [107]:
# [ScoredDoc(query_id='158491', doc_id='625257', score=15.660703104969318),

bm25 = ir_measures.read_trec_run('/content/BM25.res')
bm25_ = dict()
creators_runs = dict()
for i in list(bm25):
  st = time.time()
  query_id = i[0]
  doc_id = i[1]
  score = i[2]
  bm25_[doc_id] = score
  creators_runs[query_id] = bm25_

scores = ir_measures.calc_aggregate([P(rel=1)@10, P(rel=1)@20, AP], qrels_new, creators_runs)
et = time.time()
elapsed_time = et - st
print(scores)
print('TIME', elapsed_time)

{AP: 0.010808749000016847, P@20: 0.3000000000000005, P@10: 0.3000000000000005}
TIME 0.6766703128814697
